In [3]:
import csv
import json
from datasets import load_dataset
from gendermap import GenderMap

In [4]:
dataset = load_dataset("wikitext", "wikitext-103-v1")

In [5]:
train = dataset["train"]
validation = dataset["validation"]
test = dataset["test"]

In [6]:
len(train) + len(validation) + len(test)

1809468

In [7]:
gender_map = GenderMap(r"gendered_words/gendered_words.json")

In [15]:
def prepare_json(datasplit, start=0):
    json_data = {}
    for i, line in enumerate(datasplit["text"], start=start):
        line = line.strip().lower()
        flipped = gender_map.flip(line).strip()
        if line != flipped:
            json_data[i] = [line, flipped]
    return json_data

json_data = prepare_json(train)
json_data.update(prepare_json(validation, start=len(json_data)))
json_data.update(prepare_json(test, start=len(json_data)))

In [18]:
print(len(json_data))

437571


In [74]:
# import tarfile
# download data.tar.gz from https://github.com/SLAB-NLP/BUG/blob/main/data.tar.gz
# with tarfile.open("data.tar.gz", 'r:gz') as tar:
#     tar.extractall(path="data")

In [23]:
with open("data/data/full_BUG.csv") as file:
    reader = csv.reader(file)
    next(reader)
    counter = len(json_data)
    while True:
        try:
            row = next(reader)
            line = row[1].strip().lower()
            flipped = gender_map.flip(line).strip()
            if line != flipped:
                json_data[counter] = [line, flipped]
                counter += 1
        except StopIteration:
            break
        except (UnicodeDecodeError, IndexError):
            continue

with open("corpus.json", "w") as file:
    json.dump(json_data, file, indent=4)

In [25]:
print(len(json_data))
json_data.clear()
del json_data

525574
